In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
import dtale
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache

import simuran as smr

In [ ]:
data_path = r"D:\AllenBrainObservatory\ephys_data"

In [ ]:
def load_allen():
    data_directory = Path(data_path)
    manifest_path = data_directory / "manifest.json"
    cache = EcephysProjectCache.from_warehouse(
        manifest=manifest_path, fetch_tries=1, timeout=20000
    )
    return cache

In [ ]:
cache = load_allen()

In [ ]:
# Allen sdk help
help(cache)

In [ ]:
sessions = cache.get_session_table()

d = dtale.show(sessions)
# d.open_browser()
d

probes = cache.get_probes()

dtale.show(probes)

channels = cache.get_channels()
dtale.show(channels)

units = cache.get_units()
dtale.show(units)

In [ ]:
analysis_metrics1 = cache.get_unit_analysis_metrics_by_session_type(
    "brain_observatory_1.1",
    amplitude_cutoff_maximum=np.inf,
    presence_ratio_minimum=-np.inf,
    isi_violations_maximum=np.inf,
)

analysis_metrics2 = cache.get_unit_analysis_metrics_by_session_type(
    "functional_connectivity",
    amplitude_cutoff_maximum=np.inf,
    presence_ratio_minimum=-np.inf,
    isi_violations_maximum=np.inf,
)

all_metrics = pd.concat([analysis_metrics1, analysis_metrics2], sort=False)
dtale.show(all_metrics)

In [ ]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

In [ ]:
data_storage_directory = Path(r"D:\AllenBrainObservatory\ophys_data")
cache = VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=data_storage_directory)

In [ ]:
behavior_sessions = cache.get_behavior_session_table()
behavior_ophys_sessions = cache.get_ophys_session_table()
behavior_ophys_experiments = cache.get_ophys_experiment_table()

In [ ]:
dtale.show(behavior_ophys_sessions)

In [ ]:
dtale.show(behavior_ophys_experiments)

In [ ]:
def filter_table(table: pd.DataFrame) -> pd.DataFrame:
    """
    _summary_

    Parameters
    ----------
    table : pd.DataFrame
        _description_
    inplace : bool, optional
        _description_, by default True

    Returns
    -------
    pd.DataFrame
        _description_
    """
    genotype = "Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/wt"
    # Alternatively
    values = {
        "project_code": ["VisualBehaviorMultiscope"],
        "full_genotype": [genotype],
    }
    filters = []
    for k, v in values.items():
        filters.append(table[k].isin(v))
    full_mask = np.logical_and.reduce(np.array(filters))
    filtered_table = table[full_mask]
    return filtered_table

In [ ]:
filtered_table = filter_table(behavior_ophys_experiments)
row = filtered_table.iloc[0]
row_as_dict = row.to_dict()
row_as_dict[filtered_table.index.name] = row.name

In [ ]:
recording = smr.Recording()
recording.loader = smr.loader("allen_ophys")(cache)
recording.set_metadata(row_as_dict)
recording.available = ["signals"] # This line is silly
recording.new_load() # Set back to old load

In [ ]:
import objexplore
help(objexplore)

In [ ]:
recording.explore()
recording.show_table(recording.cell_specimen_table)